In [17]:
from pysat.formula import CNF
from pysat.solvers import Glucose3

Constants and Definitions: Constants for the number of houses, categories, and attributes are defined, along with mappings for the various attributes.

Variable Mapping: A function var is used to map a unique variable ID to each combination of house, category, and attribute.

Constraints (Clues): Clues are encoded as constraints in Conjunctive Normal Form (CNF) and added to the formula. These constraints ensure that the SAT solver finds a valid solution that satisfies the clues of Einstein's puzzle.

Solving the SAT Problem: A solver (Glucose3) is used to find a solution to the SAT problem. If a solution is found, it is interpreted based on the variable IDs.

Interpreting the Solution: The model is interpreted to map back to the attributes of the houses, forming the final solution.

In [18]:
# Constants for the problem with five houses and five categories
K = 5  # Number of houses
CATEGORIES = 5  # Categories: Nationality, Color, Drink, Pet, Cigarette
M = 5  # Number of attributes per category

# Categories and attributes
NATIONALITY, COLOR, DRINK, PET, CIGARETTE = 0, 1, 2, 3, 4
ENGLISHMAN, SPANIARD, UKRAINIAN, NORWEGIAN, JAPANESE = 0, 1, 2, 3, 4
RED, GREEN, IVORY, YELLOW, BLUE = 0, 1, 2, 3, 4
COFFEE, TEA, MILK, ORANGE_JUICE, WATER = 0, 1, 2, 3, 4
DOG, SNAILS, ZEBRA, FOX, HORSE = 0, 1, 2, 3, 4
OLD_GOLD, KOOLS, CHESTERFIELDS, LUCKY_STRIKE, PARLIAMENTS = 0, 1, 2, 3, 4


In [19]:
def var(house, category, attribute):
    return house * CATEGORIES * M + category * M + attribute + 1


In [20]:
# Create a CNF formula
formula = CNF()

# Each house has exactly one of each attribute (at least one and at most one)
for house in range(K):
    for category in range(CATEGORIES):
        formula.append([var(house, category, attr) for attr in range(M)])  # At least one attribute
        for attr1 in range(M):
            for attr2 in range(attr1 + 1, M):
                formula.append([-var(house, category, attr1), -var(house, category, attr2)])  # At most one attribute

# No two houses can have the same attribute in the same category
for category in range(CATEGORIES):
    for attr in range(M):
        for house1 in range(K):
            for house2 in range(house1 + 1, K):
                formula.append([-var(house1, category, attr), -var(house2, category, attr)])  # Uniqueness across houses


In [21]:
# The Englishman lives in the red house
for house in range(K):
    formula.append([-var(house, NATIONALITY, ENGLISHMAN), var(house, COLOR, RED)])
    formula.append([-var(house, COLOR, RED), var(house, NATIONALITY, ENGLISHMAN)])

# The Spaniard owns the dog
for house in range(K):
    formula.append([-var(house, NATIONALITY, SPANIARD), var(house, PET, DOG)])
    formula.append([-var(house, PET, DOG), var(house, NATIONALITY, SPANIARD)])

# Coffee is drunk in the green house
for house in range(K):
    formula.append([-var(house, DRINK, COFFEE), var(house, COLOR, GREEN)])
    formula.append([-var(house, COLOR, GREEN), var(house, DRINK, COFFEE)])

# The green house is immediately to the right of the ivory house
for house in range(K - 1):
    formula.append([-var(house, COLOR, IVORY), var(house + 1, COLOR, GREEN)])

# The Old Gold smoker owns snails
for house in range(K):
    formula.append([-var(house, CIGARETTE, OLD_GOLD), var(house, PET, SNAILS)])
    formula.append([-var(house, PET, SNAILS), var(house, CIGARETTE, OLD_GOLD)])

# Kools are smoked in the yellow house
for house in range(K):
    formula.append([-var(house, CIGARETTE, KOOLS), var(house, COLOR, YELLOW)])
    formula.append([-var(house, COLOR, YELLOW), var(house, CIGARETTE, KOOLS)])

# Milk is drunk in the middle house
formula.append([var(2, DRINK, MILK)])

# The Norwegian lives in the first house
formula.append([var(0, NATIONALITY, NORWEGIAN)])

# The man who smokes Chesterfields lives in the house next to the man with the fox
for house in range(K):
    if house > 0:
        formula.append([-var(house, CIGARETTE, CHESTERFIELDS), var(house - 1, PET, FOX)])
    if house < K - 1:
        formula.append([-var(house, CIGARETTE, CHESTERFIELDS), var(house + 1, PET, FOX)])

# Kools are smoked in the house next to the house where the horse is kept
for house in range(K):
    if house > 0:
        formula.append([-var(house, CIGARETTE, KOOLS), var(house - 1, PET, HORSE)])
    if house < K - 1:
        formula.append([-var(house, CIGARETTE, KOOLS), var(house + 1, PET, HORSE)])

# The Lucky Strike smoker drinks orange juice
for house in range(K):
    formula.append([-var(house, CIGARETTE, LUCKY_STRIKE), var(house, DRINK, ORANGE_JUICE)])
    formula.append([-var(house, DRINK, ORANGE_JUICE), var(house, CIGARETTE, LUCKY_STRIKE)])

# The Japanese smokes Parliaments
for house in range(K):
    formula.append([-var(house, NATIONALITY, JAPANESE), var(house, CIGARETTE, PARLIAMENTS)])
    formula.append([-var(house, CIGARETTE, PARLIAMENTS), var(house, NATIONALITY, JAPANESE)])

# The Norwegian lives next to the blue house
for house in range(K):
    if house > 0:
        formula.append([-var(house, NATIONALITY, NORWEGIAN), var(house - 1, COLOR, BLUE)])
    if house < K - 1:
        formula.append([-var(house, NATIONALITY, NORWEGIAN), var(house + 1, COLOR, BLUE)])


In [22]:
print("Formula:")
print(formula)

Formula:
CNF(from_string="p cnf 125 625\n1 2 3 4 5 0\n-1 -2 0\n-1 -3 0\n-1 -4 0\n-1 -5 0\n-2 -3 0\n-2 -4 0\n-2 -5 0\n-3 -4 0\n-3 -5 0\n-4 -5 0\n6 7 8 9 10 0\n-6 -7 0\n-6 -8 0\n-6 -9 0\n-6 -10 0\n-7 -8 0\n-7 -9 0\n-7 -10 0\n-8 -9 0\n-8 -10 0\n-9 -10 0\n11 12 13 14 15 0\n-11 -12 0\n-11 -13 0\n-11 -14 0\n-11 -15 0\n-12 -13 0\n-12 -14 0\n-12 -15 0\n-13 -14 0\n-13 -15 0\n-14 -15 0\n16 17 18 19 20 0\n-16 -17 0\n-16 -18 0\n-16 -19 0\n-16 -20 0\n-17 -18 0\n-17 -19 0\n-17 -20 0\n-18 -19 0\n-18 -20 0\n-19 -20 0\n21 22 23 24 25 0\n-21 -22 0\n-21 -23 0\n-21 -24 0\n-21 -25 0\n-22 -23 0\n-22 -24 0\n-22 -25 0\n-23 -24 0\n-23 -25 0\n-24 -25 0\n26 27 28 29 30 0\n-26 -27 0\n-26 -28 0\n-26 -29 0\n-26 -30 0\n-27 -28 0\n-27 -29 0\n-27 -30 0\n-28 -29 0\n-28 -30 0\n-29 -30 0\n31 32 33 34 35 0\n-31 -32 0\n-31 -33 0\n-31 -34 0\n-31 -35 0\n-32 -33 0\n-32 -34 0\n-32 -35 0\n-33 -34 0\n-33 -35 0\n-34 -35 0\n36 37 38 39 40 0\n-36 -37 0\n-36 -38 0\n-36 -39 0\n-36 -40 0\n-37 -38 0\n-37 -39 0\n-37 -40 0\n-38 -39 0\n-3

In [23]:
solver = Glucose3()
solver.append_formula(formula)
solution_found = solver.solve()
model = solver.get_model() if solution_found else None

# Print the SAT solver's output
print("Solution Found:", solution_found)
if solution_found:
    print("Model (first 10 literals):", model[:10])
else:
    print("Model is None.")


Solution Found: True
Model (first 10 literals): [-1, -2, -3, 4, -5, -6, -7, -8, 9, -10]


In [24]:
# Define the category names and attribute names for interpretation
categories_names = ["Nationality", "Color", "Drink", "Pet", "Cigarette"]
attributes_names = [
    ["Englishman", "Spaniard", "Ukrainian", "Norwegian", "Japanese"],
    ["Red", "Green", "Ivory", "Yellow", "Blue"],
    ["Coffee", "Tea", "Milk", "Orange Juice", "Water"],
    ["Dog", "Snails", "Zebra", "Fox", "Horse"],
    ["Old Gold", "Kools", "Chesterfields", "Lucky Strike", "Parliaments"],
]

if solution_found:
    # Interpret the solution
    solution = []
    for house in range(K):
        house_info = {}
        for category in range(CATEGORIES):
            for attr in range(M):
                variable_id = var(house, category, attr)
                if variable_id in model: # Check if the variable ID is in the model as a positive literal
                    house_info[categories_names[category]] = attributes_names[category][attr]
        solution.append(house_info)
    print("Final Solution:", solution)
else:
    print("No solution found.")



Final Solution: [{'Nationality': 'Norwegian', 'Color': 'Yellow', 'Drink': 'Water', 'Pet': 'Zebra', 'Cigarette': 'Kools'}, {'Nationality': 'Ukrainian', 'Color': 'Blue', 'Drink': 'Orange Juice', 'Pet': 'Horse', 'Cigarette': 'Lucky Strike'}, {'Nationality': 'Englishman', 'Color': 'Red', 'Drink': 'Milk', 'Pet': 'Snails', 'Cigarette': 'Old Gold'}, {'Nationality': 'Japanese', 'Color': 'Green', 'Drink': 'Coffee', 'Pet': 'Fox', 'Cigarette': 'Parliaments'}, {'Nationality': 'Spaniard', 'Color': 'Ivory', 'Drink': 'Tea', 'Pet': 'Dog', 'Cigarette': 'Chesterfields'}]
